In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic3/titanic3.xls


**Objective : Predicting Survival based on varaibles pclass,sex,gender,fare.**

## Importing Libraries

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np

**Linearly non - separable?	Decision Tree**

**Categorical Data type	Decision Tree**

**Continuous Data type	Logistic Regression**

**Skewed	 Decision Tree, or give high weight to minority class in Logistic Regression**

**Outlier	Decision Tree, or remove outlier for Logistic Regression**

**Lots of Missing values	Decision Tree**

## Loading Dataset

In [3]:
# xlrd is a library for reading data and formatting information from 
# Excel files in the historical .xls format.
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 3.2 MB/s eta 0:00:00


In [4]:
df = pd.read_excel('/kaggle/input/titanic3/titanic3.xls')

In [5]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


## Data Preprocessing

**Since, We are Predicting Survival based on varaibles pclass,sex,gender,fare; so we will drop rest of the columns**

In [6]:
df.drop(columns=['name','sibsp','parch','ticket','cabin','embarked','boat','body','home.dest'],inplace=True)

In [7]:
df.isna().sum()

pclass        0
survived      0
sex           0
age         263
fare          1
dtype: int64

In [8]:
df.dtypes

pclass        int64
survived      int64
sex          object
age         float64
fare        float64
dtype: object

**Replacing Null Values**

In [9]:
df.groupby('sex')['age'].mean()

sex
female    28.687071
male      30.585233
Name: age, dtype: float64

In [10]:
age_mean = df.groupby('sex',as_index =False)['age'].mean()
age_mean

,sex,age
0,female,28.687071
1,male,30.585233


In [11]:
df.loc[df['sex'] == 'female', 'age'] = df.loc[df['sex'] == 'female', 'age'].fillna(age_mean['age'][0])

In [12]:
df.loc[df['sex'] == 'male', 'age'] = df.loc[df['sex'] == 'male', 'age'].fillna(age_mean['age'][1])

In [13]:
df.dropna(axis=0,inplace =True) #Dropping 1 Fare Null Value

In [14]:
df.head()

,pclass,survived,sex,age,fare
0,1,1,female,29.0000,211.3375
1,1,1,male,0.9167,151.5500
2,1,0,female,2.0000,151.5500
3,1,0,male,30.0000,151.5500
4,1,0,female,25.0000,151.5500


## Preparing Data For Training of Model

In [15]:
dummies = pd.get_dummies(df['sex'],dtype=int)

In [16]:
df_dummies = pd.concat([df,dummies],axis='columns')

In [17]:
df_dummies.drop(columns='sex',inplace = True)

In [18]:
df_dummies.head()

,pclass,survived,age,fare,female,male
0,1,1,29.0000,211.3375,1,0
1,1,1,0.9167,151.5500,0,1
2,1,0,2.0000,151.5500,1,0
3,1,0,30.0000,151.5500,0,1
4,1,0,25.0000,151.5500,1,0


In [19]:
features = df_dummies[['pclass','age','fare','female','male']]

In [20]:
target = df_dummies['survived']

In [21]:
x_train,x_test,y_train,y_test = train_test_split(features,target,test_size=0.2)

## Training the Decision Tree Model

In [22]:
model = tree.DecisionTreeClassifier()

In [23]:
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [24]:
model.score(x_test,y_test)

0.7595419847328244

In [25]:
model.predict([[1,20,400,1,0]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])

In [26]:
model.predict([[3,20,100,1,0]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])

In [27]:
model.predict([[3,20,100,0,1]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])